In [22]:
using AbstractAlgebra
using AlgebraicSolving

In [46]:
R, vars = polynomial_ring(GF(65521), ["x$i" for i in 1:4])
F = AlgebraicSolving.cyclic(R)
groebner_basis(F)

7-element Vector{fpMPolyRingElem}:
 x1 + x2 + x3 + x4
 x2^2 + 2*x2*x4 + x4^2
 x2*x3^2 + x3^2*x4 + 65520*x2*x4^2 + 65520*x4^3
 x2*x3*x4^2 + x3^2*x4^2 + 65520*x2*x4^3 + x3*x4^3 + 65520*x4^4 + 65520
 x2*x4^4 + x4^5 + 65520*x2 + 65520*x4
 x3^3*x4^2 + x3^2*x4^3 + 65520*x3 + 65520*x4
 x3^2*x4^4 + x2*x3 + 65520*x2*x4 + x3*x4 + 65519*x4^2

In [49]:
R, (x, y, u, v, l) = polynomial_ring(GF(65521),["x","y","u","v","l"], ordering=:degrevlex)
F = [x^2+y^2-1, 2*x*u+2*y*v]
F_hom = AlgebraicSolving._homogenize(F)
sol = sig_groebner_basis(F_hom)

3-element Vector{Tuple{Tuple{Int64, fpMPolyRingElem}, fpMPolyRingElem}}:
 ((1, 1), x1^2 + x2^2 + 65520*x6^2)
 ((2, 1), x1*x3 + x2*x4)
 ((2, x1), x2^2*x3 + 65520*x1*x2*x4 + 65520*x3*x6^2)

In [35]:
function partial(q, ps)
    partial_q = 0
    for (i, pi) in enumerate(ps)
        partial_q =  partial_q + pi * derivative(q, i)
    end
    return partial_q
end;

In [77]:
function naive_algorithm(q, ps)
    i = 0 
    S = q
    g = q
    G = groebner_basis(Ideal(S))
    while true
        g = [partial(gi, ps) for gi in g]
        g = [AlgebraicSolving.normal_form(gi, Ideal(G)) for gi in g]
        if all(g .== 0)
            return G
        else
            append!(S, g)
            G = groebner_basis(Ideal(S))
        end
    end    
end;

In [80]:
R, (x, y, u, v, l) = polynomial_ring(GF(65521),["x","y","u","v","l"], ordering=:degrevlex)
ps = [
    u,
    v, 
    l*x,
    l*y - 1,
    0
] 

q = [x^2 + y^2 - 1]
G = naive_algorithm(q, ps)
println(G)

fpMPolyRingElem[v, y*l + 65520, u^2 + 65520*y + l, y*u + 65520*u*l, x*u, y^2 + l^2 + 65519, x*y + 65520*x*l, x^2 + 65520*l^2 + 1, l^3 + y + 65519*l, u*l^2 + 65520*u, x*l^2 + 65520*x]
